In [1]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML
import re
from collections import Counter

from isoweek import Week
from pandas_summary import DataFrameSummary
import operator
from matplotlib import pyplot as plt
import pickle 
# import keras 
import multiprocessing as mp
import datetime as dt
from tqdm import tqdm_notebook
import multiprocessing
import scipy.stats as st

from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder, Imputer, StandardScaler


# import seaborn as sns

# from tsfresh import extract_features
# from tsfresh.utilities.dataframe_functions import make_forecasting_frame
# from sklearn.ensemble import AdaBoostRegressor
# from tsfresh.utilities.dataframe_functions import impute
# from tsfresh.utilities.dataframe_functions import roll_time_series



%matplotlib inline

### load data, preprocess

- remove spikes (winsorize)
- replace na by median (todo: avg of earliest available values in past and fututre)


In [3]:
# train = pd.read_csv('data/train_1.csv')

In [3]:
def winsorize_row(row):
#     print (type(row))
    row_copy = row
    median = np.median(row[1:])
    winsorized = pd.Series(st.mstats.winsorize( row[1:], limits=[0.05, 0.05]))
    if median> 0:
        winsorized.fillna(median, inplace=True)
    else:
        winsorized.fillna(0, inplace=True)
        
    row_copy[1:] =    winsorized
    return row_copy

In [4]:
# train.iloc[0:10,1:] = train.iloc[0:10].apply(winsorize_row,axis=1)
# s = train.iloc[0]
# s[1:].mean()
# s.iloc[1:] = st.mstats.winsorize( s[1:], limits=[0.05, 0.05])

In [5]:
sample = train[['Page']+list(train.columns[-50:])]

In [6]:
sample_winsorized = sample.apply(winsorize_row,axis=1)

In [7]:
sample_winsorized.to_csv("data/cache/sample_1.csv", index=False)

In [4]:
sample_winsorized = pd.read_csv("data/cache/sample_1.csv")

In [5]:
sample_winsorized.head(2)

,Page,2016-11-12,2016-11-13,2016-11-14,2016-11-15,2016-11-16,2016-11-17,2016-11-18,2016-11-19,2016-11-20,...,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,13.0,10.0,15.0,14.0,12.0,10.0,11.0,10.0,42.0,...,32.0,63.0,15.0,26.0,14.0,20.0,22.0,19.0,18.0,20.0
1,2PM_zh.wikipedia.org_all-access_spider,12.0,11.0,14.0,28.0,23.0,20.0,9.0,12.0,11.0,...,17.0,42.0,28.0,15.0,9.0,30.0,52.0,45.0,26.0,20.0


In [104]:
# sample_winsorized.isnull().sum(axis=1)
train.shape, sample_winsorized.shape

((145063, 551), (145063, 51))

In [9]:
key_1 = pd.read_csv('data/key_1.csv')
# train = pd.read_csv('data/train_1.csv')
ss_1 = pd.read_csv('data/sample_submission_1.csv')


In [3]:
display(key_1.tail(2))
display(train_1.head(2))
display(ss_1.head(2))

,Page,Id
8703778,龙生九子_zh.wikipedia.org_mobile-web_all-agents_20...,24a1dfb06c10
8703779,龙生九子_zh.wikipedia.org_mobile-web_all-agents_20...,add681d54216


,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,32.0,63.0,15.0,26.0,14.0,20.0,22.0,19.0,18.0,20.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,17.0,42.0,28.0,15.0,9.0,30.0,52.0,45.0,26.0,20.0


,Id,Visits
0,bf4edcf969af,0
1,929ed2bf52b9,0


In [4]:
key_1.Page[0]

'!vote_en.wikipedia.org_all-access_all-agents_2017-01-01'

In [5]:
ids = key_1.Id.values
pages = key_1.Page.values

## Features

- language 
- topic 
- day of week, month,.. 
    - mean since last week 
- holidays, days since holidays 
- days since a peak

In [6]:
def add_datepart(df):
    df.Date = pd.to_datetime(df.date)
    df["Year"] = df.date.dt.year
    df["Month"] = df.date.dt.month
    df["Week"] = df.date.dt.week
    df["Day"] = df.date.dt.day
    df['Weekend'] = ((df.date.dt.dayofweek) // 5 == 1).astype(float)
    df['Weekday'] = df['date'].apply(lambda x: x.weekday())
    

In [7]:
def join_df(left, right, left_on, right_on=None):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", "_y"))


In [8]:
# train_flattened = pd.melt(train[list(train.columns[-50:])+['Page']], id_vars='Page', var_name='date', value_name='Visits')
# train_flattened.tail(2)

train_flattened = pd.melt(sample_winsorized, id_vars='Page', var_name='date', value_name='Visits')
train_flattened.tail(2)

,Page,date,Visits
7253148,Hasta_el_último_hombre_es.wikipedia.org_all-ac...,2016-12-31,0.0
7253149,Francisco_el_matemático_(serie_de_televisión_d...,2016-12-31,0.0


In [152]:
# train_flattened.set_index("date", inplace=True)

In [9]:
train_flattened['date'] = train_flattened['date'].astype('datetime64[ns]')


In [10]:
page_details = train_flattened.Page.str.extract(r'(?P<topic>.*)\_(?P<lang>.*).wikipedia.org\_(?P<access>.*)\_(?P<type>.*)')


/home/ubuntu/anaconda2/envs/pxx/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


In [16]:
page_details.shape, train_flattened.shape, train.shape

((7253150, 4), (7253150, 3), (145063, 551))

In [17]:
page_details[0:2]


,topic,lang,access,type
0,2NE1,zh,all-access,spider
1,2PM,zh,all-access,spider


In [11]:
train_df = pd.concat([page_details, train_flattened], axis=1)

In [12]:
add_datepart(train_df)

In [13]:
train_df.to_csv("data/cache/train_df_t1.csv")

In [26]:
# df_median = pd.DataFrame(train_flattened.groupby(['Page'])['Visits'].median())
# df_median.columns = ['median']

# # Average by page
# df_mean = pd.DataFrame(train_flattened.groupby(['Page'])['Visits'].mean())
# df_mean.columns = ['mean']


In [27]:
# train_df = train_df.set_index('Page').join(df_mean).join(df_median)


In [14]:
train_df.head(2)

,topic,lang,access,type,Page,date,Visits,Year,Month,Week,Day,Weekend,Weekday
0,2NE1,zh,all-access,spider,2NE1_zh.wikipedia.org_all-access_spider,2016-11-12,13.0,2016,11,45,12,1.0,5
1,2PM,zh,all-access,spider,2PM_zh.wikipedia.org_all-access_spider,2016-11-12,12.0,2016,11,45,12,1.0,5


In [28]:
train_df.columns

Index(['topic', 'lang', 'access', 'type', 'Page', 'date', 'Visits', 'Year',
       'Month', 'Week', 'Day', 'Weekend', 'Weekday'],
      dtype='object')

In [46]:
train_df.reset_index(drop=False,inplace=True)

#### Durations


It is common when working with time series data to extract data that explains relationships across rows as opposed to columns, e.g.:
Running averages
Time until next event
Time since last event

In [15]:
df = train_df.set_index("date")


In [16]:
bwd = df[['Page',"Visits"]].sort_index().groupby("Page").rolling(7, min_periods=1).sum()


In [17]:
fwd = df[['Page',"Visits"]].sort_index(ascending=False).groupby("Page").rolling(7, min_periods=1).sum()



In [18]:
bwd.drop('Page',1,inplace=True)
bwd.reset_index(inplace=True)

fwd.drop('Page',1,inplace=True)
fwd.reset_index(inplace=True)

In [19]:
bwd.head(2)

,Page,date,Visits
0,!vote_en.wikipedia.org_all-access_all-agents,2016-11-12,2.0
1,!vote_en.wikipedia.org_all-access_all-agents,2016-11-13,4.0


In [ ]:
#### ToDo

# fwd = df[['Store']+columns].sort_index(ascending=False
# fwd.drop('Store',1,inplace=True)
# fwd.reset_index(inplace=True)

In [20]:
df.reset_index(inplace=True)


In [21]:
df = df.merge(bwd, 'left', ['date', 'Page'], suffixes=['', '_bw'])
df = df.merge(fwd, 'left', ['date', 'Page'], suffixes=['', '_fw'])


In [22]:
df.head(2)

,date,topic,lang,access,type,Page,Visits,Year,Month,Week,Day,Weekend,Weekday,Visits_bw,Visits_fw
0,2016-11-12,2NE1,zh,all-access,spider,2NE1_zh.wikipedia.org_all-access_spider,13.0,2016,11,45,12,1.0,5,13.0,85.0
1,2016-11-12,2PM,zh,all-access,spider,2PM_zh.wikipedia.org_all-access_spider,12.0,2016,11,45,12,1.0,5,12.0,117.0


In [23]:
df.to_csv('data/cache/df_temp1.csv',index=False)


## create features

This dictionary maps categories to embedding dimensionality. 
In generally, categories we might expect to be conceptually more complex have larger dimension.


In [24]:
df.columns

Index(['date', 'topic', 'lang', 'access', 'type', 'Page', 'Visits', 'Year',
       'Month', 'Week', 'Day', 'Weekend', 'Weekday', 'Visits_bw', 'Visits_fw'],
      dtype='object')

In [25]:
df.head(2)

,date,topic,lang,access,type,Page,Visits,Year,Month,Week,Day,Weekend,Weekday,Visits_bw,Visits_fw
0,2016-11-12,2NE1,zh,all-access,spider,2NE1_zh.wikipedia.org_all-access_spider,13.0,2016,11,45,12,1.0,5,13.0,85.0
1,2016-11-12,2PM,zh,all-access,spider,2PM_zh.wikipedia.org_all-access_spider,12.0,2016,11,45,12,1.0,5,12.0,117.0


In [29]:
#'StateHoliday': 3,
cat_var_dict = {'topic': 100,'lang':6, 'access':3, 'type':3, 'Weekday': 6, 'Year': 2, 'Month': 6,
'Day': 10, 'Week': 2, 'Weekend': 1}

In [30]:
cat_vars = [o[0] for o in 
            sorted(cat_var_dict.items(), key=operator.itemgetter(1), reverse=True)]

In [31]:
cat_vars

['topic',
 'Day',
 'lang',
 'Weekday',
 'Month',
 'access',
 'type',
 'Year',
 'Week',
 'Weekend']

likewise for continuous

In [32]:
contin_vars = ['Visits', 'Visits_bw',"Visits_fw"]

In [33]:
for v in contin_vars: df.loc[df[v].isnull(), v] = 0.0
for v in cat_vars: df.loc[df[v].isnull(), v] = ""


In [34]:
cat_maps = [(o, LabelEncoder()) for o in cat_vars]
contin_maps = [([o], StandardScaler()) for o in contin_vars]

In [36]:
cat_maps

[('topic', LabelEncoder()),
 ('Day', LabelEncoder()),
 ('lang', LabelEncoder()),
 ('Weekday', LabelEncoder()),
 ('Month', LabelEncoder()),
 ('access', LabelEncoder()),
 ('type', LabelEncoder()),
 ('Year', LabelEncoder()),
 ('Week', LabelEncoder()),
 ('Weekend', LabelEncoder())]

In [37]:
cat_mapper = DataFrameMapper(cat_maps)
cat_cols = len(cat_map_fit.features)
cat_cols

10

In [38]:
contin_mapper = DataFrameMapper(contin_maps)
contin_map_fit = contin_mapper.fit(df)
contin_cols = len(contin_map_fit.features)
contin_cols

3

In [41]:
pickle.dump(contin_map_fit, open('data/cache/contin_maps.pickle', 'wb'))
pickle.dump(cat_map_fit, open('data/cache/cat_maps.pickle', 'wb'))

In [42]:
[len(o[1].classes_) for o in cat_map_fit.features]


[42213, 31, 8, 7, 2, 4, 3, 1, 8, 2]

In [21]:
contin_map_fit = pickle.load(open('data/cache/contin_maps.pickle', 'rb'))
cat_map_fit = pickle.load(open('data/cache/cat_maps.pickle', 'rb'))

In [10]:
df = pd.read_csv('data/cache/df_temp1.csv')


C:\Users\Armin\Anaconda2\envs\cx\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Sample data

In [43]:
n = len(df)
n

7253150

In [88]:
# samp_size = 100000
# np.random.seed(42)
# idxs = sorted(np.random.choice(n, samp_size, replace=False))

In [90]:
# df_sample = df.iloc[idxs].set_index("date")


In [44]:
samp_size = n
train_ratio = 0.9
train_size = int(samp_size * train_ratio)
joined_samp = df.set_index("date")


In [45]:
train_size

6527835

In [46]:
joined_valid = joined_samp[train_size:]
joined_train = joined_samp[:train_size]
len(joined_valid), len(joined_train)

(725315, 6527835)

In [47]:
def contin_preproc(dat):
    return contin_map_fit.transform(dat).astype(np.float32)
def cat_preproc(dat):
    return cat_map_fit.transform(dat).astype(np.int64)

In [48]:
cat_map_train = cat_preproc(joined_train)
cat_map_valid = cat_preproc(joined_valid)

In [49]:
contin_map_train = contin_preproc(joined_train)
contin_map_valid = contin_preproc(joined_valid)

#### targets

In [50]:
y_train_orig = joined_train.Visits
y_valid_orig = joined_valid.Visits

In [51]:
y_train = np.log1p(y_train_orig)
y_valid = np.log1p(y_valid_orig)

In [58]:
y_train_orig.to_csv("data/cache/y_train_orig_1.csv",index=False)
y_valid_orig.to_csv("data/cache/y_valid_orig_1.csv",index=False)

In [52]:
from numba import jit
import math



def rmspe(y_pred, targ = y_valid_orig):
    pct_var = (targ - y_pred)/targ
    return math.sqrt(np.square(pct_var).mean())

def log_max_inv(preds, mx = 1):
    return np.exp(preds * mx)-1


def normalize_inv(preds):
    return preds * ystd + ymean





@jit
def smape_fast(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true[i]
        b = math.fabs(y_pred[i])
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out


### Create models

In [59]:
def split_cols(arr): return np.hsplit(arr,arr.shape[1])
def cat_map_info(feat): return feat[0], len(feat[1].classes_)


In [60]:
map_train = split_cols(cat_map_train) + [contin_map_train]
map_valid = split_cols(cat_map_valid) + [contin_map_valid]

# map_train = split_cols(cat_map_train) + split_cols(contin_map_train)
# map_valid = split_cols(cat_map_valid) + split_cols(contin_map_valid)

In [64]:
pickle.dump(map_train, open('data/cache/map_train_1.pickle', 'wb'))

In [65]:
pickle.dump(map_valid, open('data/cache/map_valid_1.pickle', 'wb'))

In [66]:
len(map_train)

11

In [67]:
def my_init(scale):
    return lambda shape, name=None: keras.initializers.uniform(shape, scale=scale, name=name)

def emb_init(shape, name=None): 
#     return initializations.uniform(shape, scale=2/(shape[1]+1), name=name)
    return keras.initializers.uniform(shape, scale=2/(shape[1]+1), name=name)

In [68]:
def get_emb(feat):
    name, c = cat_map_info(feat)
    #c2 = cat_var_dict[name]
    c2 = (c+1)//2
#     if c2>50: c2=50
    inp = Input((1,), dtype='int64', name=name+'_in')
    # , W_regularizer=l2(1e-6)
    u = Flatten(name=name+'_flt')(Embedding(c, c2, input_length=1, )(inp)) #init=emb_init
#     u = Flatten(name=name+'_flt')(Embedding(c, c2, input_length=1)(inp))
    return inp,u

In [69]:
def get_contin(feat):
    name = feat[0][0]
    inp = Input((1,), name=name+'_in')
    return inp, Dense(1, name=name+'_d', init=my_init(1.))(inp)

In [70]:
from keras.layers import Input,Dense,Dropout,Flatten,Embedding, merge
from keras.models import Model, Sequential

# from keras import initializations


Using TensorFlow backend.


In [71]:
contin_inp = Input((contin_cols,), name='contin')
contin_out = Dense(contin_cols*10, activation='relu', name='contin_d')(contin_inp)

Now we can put them together. Given the inputs, continuous and categorical embeddings, we're going to concatenate all of them.

Next, we're going to pass through some dropout, then two dense layers w/ ReLU activations, then dropout again, then the sigmoid activation we mentioned earlier.

In [72]:
embs = [get_emb(feat) for feat in cat_map_fit.features]
#conts = [get_contin(feat) for feat in contin_map_fit.features]
#contin_d = [d for inp,d in conts]
x = merge([emb for inp,emb in embs] + [contin_out], mode='concat')
#x = merge([emb for inp,emb in embs] + contin_d, mode='concat')

x = Dropout(0.02)(x)
x = Dense(1000, activation='relu', kernel_initializer='uniform')(x)
x = Dense(500, activation='relu', kernel_initializer='uniform')(x)
x = Dropout(0.2)(x)
x = Dense(1, activation='sigmoid')(x)



/home/ubuntu/anaconda2/envs/pxx/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/ubuntu/anaconda2/envs/pxx/lib/python3.6/site-packages/keras/legacy/layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [73]:
model = Model([inp for inp,emb in embs] + [contin_inp], x)
#model = Model([inp for inp,emb in embs] + [inp for inp,d in conts], x)


In [74]:
model.compile('adam', 'mean_absolute_error')
#model.compile(Adam(), 'mse')

###  Start training¶


In [ ]:
hist = model.fit(map_train, y_train, batch_size=128, epochs=25,
                 verbose=0, validation_data=(map_valid, y_valid))

/home/ubuntu/anaconda2/envs/pxx/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app
/home/ubuntu/anaconda2/envs/pxx/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:90: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 890989791 elements. This may consume a large amount of memory.
  "This may consume a large amount of memory." % num_elements)
